In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",   # IMPORTANT: new correct name
    google_api_key="your key",
    temperature=0.7
)



In [34]:

from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from urllib.parse import quote_plus

db_user = "root"
db_password = "password"
encoded_password = quote_plus(db_password)
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{encoded_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [35]:
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain import hub
from langchain.chains import create_sql_query_chain

In [36]:
sql_chain = create_sql_query_chain(llm, db)

In [37]:
execute_query = QuerySQLDataBaseTool(db=db)

In [38]:
qns1 = sql_chain.invoke({'question':"How many t-shirts do we have left for nike in extra small size and white color?"})
qns1

"SQLQuery: SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White';"

In [39]:
qns1[10:]

"SELECT `stock_quantity` FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White';"

In [40]:
output = execute_query.invoke({"query":qns1[10:]})
output

'[(48,)]'

In [41]:
qns2 = sql_chain.invoke({"question":"How much is the price of the inventory for all small size t-shirts?"})
print(qns2)
output2 = execute_query.invoke({"query":qns2[10:]})


SQLQuery: SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `size` = 'S';


In [42]:
output2

"[(Decimal('17423'),)]"

In [43]:
query3 = sql_chain.invoke({"question":"If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})
output3 = execute_query.invoke({"query":query3[10:]})
output3

"[(Decimal('27534.000000'),)]"

In [44]:
def get_output(query):
    qu = sql_chain.invoke({"question":query})
    output = execute_query.invoke({"query":qu[10:]})
    return output

In [45]:
qns4 = get_output("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")
qns4

"[(Decimal('28105'),)]"

In [46]:
qns5 = get_output("How many white color Levi's t shirts we have available?")
qns5

"[(Decimal('252'),)]"

In [47]:
qns6 = get_output("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
qns6

"[(Decimal('252'),)]"